<a href="https://colab.research.google.com/github/MuhammadShaheer-khan/Python-For-Finance-Stock-Analysis/blob/master/Stock_Price_Predict_Using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Tjis model predict stock price using machine learning

In [2]:
pip install quandl

In [3]:
# import Libraries
from sklearn.svm import SVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
import quandl
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [4]:
# Get the stock data
df = quandl.get("wiki/FB")

# Take a look at the data
print(df.head())

             Open   High    Low  ...  Adj. Low  Adj. Close  Adj. Volume
Date                             ...                                   
2012-05-18  42.05  45.00  38.00  ...     38.00     38.2318  573576400.0
2012-05-21  36.53  36.66  33.00  ...     33.00     34.0300  168192700.0
2012-05-22  32.61  33.59  30.94  ...     30.94     31.0000  101786600.0
2012-05-23  31.37  32.50  31.36  ...     31.36     32.0000   73600000.0
2012-05-24  32.95  33.21  31.77  ...     31.77     33.0300   50237200.0

[5 rows x 12 columns]


In [5]:
# Get the adj close price
df = df[["Adj. Close"]]
df.head()

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [6]:
# A variable for prediction " n " days out into the future
forecast_out = 30

# Create another column (the target or dependent variable) shifted " n " units up
df["Predictions"] = df[["Adj. Close"]].shift(-forecast_out)

df.head()

,Adj. Close,Predictions
Date,,
2012-05-18,38.2318,30.771
2012-05-21,34.0300,31.200
2012-05-22,31.0000,31.470
2012-05-23,32.0000,31.730
2012-05-24,33.0300,32.170


In [7]:
df.tail()

,Adj. Close,Predictions
Date,,
2018-03-21,169.39,NaN
2018-03-22,164.89,NaN
2018-03-23,159.39,NaN
2018-03-26,160.06,NaN
2018-03-27,152.19,NaN


In [8]:
### Create the independent data set (x) ###
# Convert the dataframe to numpy array
X = np.array(df.drop(["Predictions"],1))

# Remove the last " n " row
X = X[:-forecast_out]
print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


In [9]:
### Create the dependent data set (y) ###
# Convert the dataframe to numpy array (the the values including the NaN's)
y = np.array(df["Predictions"])

y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [10]:
# split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [11]:
# Create and train the Support Vector Machine (regressor)
svr_rbf = SVR(kernel="rbf", C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [12]:
# Testing model: Score returns the coefficient of determination R^2 of the prediction.
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confident: ", svm_confidence)

svm confident:  0.9832636626856024


In [14]:
# Create and train the linear Regressor Model
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
# Testing model: Score returns the coefficient of determination R^2 of the prediction.
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("linear regression confident: ", svm_confidence)

linear regression confident:  0.9832636626856024


In [17]:
# Set x_forecast equal to the last 30 rows of the orignal data set from Adj. close
x_forecast = np.array(df.drop(["Predictions"], 1))[-forecast_out:]
print(x_forecast)

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


In [18]:
# print the Lr predictions for the next " n " days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[177.15093698 183.58792863 184.032556   181.40521247 180.04101487
 181.96099668 183.05235476 187.39757675 189.0548242  185.54833111
 182.37530854 179.97027869 180.65743008 184.47718336 183.85066298
 187.82199378 186.43758584 189.35797923 188.88303636 185.97274814
 188.30704181 187.97357129 189.21650688 176.55473211 172.09835327
 173.35139403 168.8040687  163.24622662 163.92327283 155.97050607]


In [19]:
# print the SVR predictions for the next " n " days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[174.66982008 178.89526338 178.42424251 179.32477714 172.54841754
 181.58343007 180.4941568  185.76810913 179.48082906 184.79672505
 181.97074329 172.40201846 174.96776494 179.22854151 178.4762716
 183.72711256 187.8327453  179.50591001 179.68696551 186.78643653
 181.43376704 182.9728903  179.43303536 174.78492229 170.84475033
 173.14154841 171.86674523 167.04705258 168.3461349  162.40672298]
